In [2]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
import tensorflow as tf

# Basic RNNs 

In [4]:
n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

init = tf.global_variables_initializer()

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
import numpy as np

X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # t = 0
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # t = 1

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [6]:
Y0_val

array([[-0.9721628 , -0.846264  ,  0.625283  , -0.92134786,  0.9975373 ],
       [-0.9977104 , -0.9999528 ,  0.9994099 , -1.        ,  0.99999976],
       [-0.99981385, -1.        ,  0.9999993 , -1.        ,  1.        ],
       [ 0.99990773, -0.9924926 , -0.49361616, -1.        ,  0.19812426]],
      dtype=float32)

In [7]:
Y1_val

array([[-0.999879  , -0.9999998 ,  0.99999183, -1.        ,  1.        ],
       [-0.9971111 ,  0.99256384, -0.9957139 , -0.22613814,  0.28785455],
       [-0.9986549 , -0.99948007,  0.9934059 , -1.        ,  0.99998003],
       [ 0.22712618, -0.87831664, -0.21702453, -0.99999493,  0.8590004 ]],
      dtype=float32)

# Using `static_rnn()`

In [8]:
reset_graph()

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1], 
                                                 dtype=tf.float32)
Y0, Y1 = output_seqs

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API


In [9]:
init = tf.global_variables_initializer()

X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # t = 0
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # t = 1

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [10]:
Y0_val

array([[ 0.30741334, -0.32884315, -0.6542847 , -0.9385059 ,  0.52089024],
       [ 0.99122757, -0.9542541 , -0.7518079 , -0.9995208 ,  0.9820235 ],
       [ 0.9999268 , -0.99783254, -0.8247353 , -0.9999963 ,  0.99947774],
       [ 0.996771  , -0.68750614,  0.8419969 ,  0.9303911 ,  0.8120684 ]],
      dtype=float32)

In [11]:
Y1_val

array([[ 0.99998885, -0.99976057, -0.0667929 , -0.9999803 ,  0.99982214],
       [-0.6524943 , -0.51520866, -0.37968948, -0.5922594 , -0.08968379],
       [ 0.99862397, -0.99715203, -0.03308626, -0.9991566 ,  0.9932902 ],
       [ 0.99681675, -0.9598194 ,  0.39660627, -0.8307606 ,  0.79671973]],
      dtype=float32)

### Packing sequences

In [12]:
reset_graph()

In [13]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [14]:
reset_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2]))

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.nn.static_rnn(basic_cell, X_seqs,
                                      dtype=tf.float32)

outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])

In [15]:
init = tf.global_variables_initializer()

In [16]:
X_batch = np.array([
        # t = 0      t = 1 
        [[0, 1, 2], [9, 8, 7]], # instance 1
        [[3, 4, 5], [0, 0, 0]], # instance 2
        [[6, 7, 8], [6, 5, 4]], # instance 3
        [[9, 0, 1], [3, 2, 1]], # instance 4
    ])

with tf.Session() as sess:
    init.run()
    outputs_val = sess.run(outputs, feed_dict={X: X_batch})

In [17]:
outputs_val

array([[[-0.45652324, -0.68064123,  0.40938237,  0.63104504,
         -0.45732826],
        [-0.9428799 , -0.9998869 ,  0.94055814,  0.9999985 ,
         -0.9999997 ]],

       [[-0.8001535 , -0.9921827 ,  0.7817797 ,  0.9971032 ,
         -0.9964609 ],
        [-0.637116  ,  0.11300927,  0.5798437 ,  0.4310559 ,
         -0.6371699 ]],

       [[-0.93605185, -0.9998379 ,  0.9308867 ,  0.9999815 ,
         -0.99998295],
        [-0.9165386 , -0.9945604 ,  0.896054  ,  0.99987197,
         -0.9999751 ]],

       [[ 0.9927369 , -0.9981933 , -0.55543643,  0.9989031 ,
         -0.9953323 ],
        [-0.02746338, -0.73191994,  0.7827872 ,  0.9525682 ,
         -0.9781773 ]]], dtype=float32)

In [18]:
outputs_val.shape

(4, 2, 5)

# Using `dynamic_rnn()`

In [19]:
reset_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

init = tf.global_variables_initializer()

X_batch = np.array([
        # t = 0      t = 1 
        [[0, 1, 2], [9, 8, 7]], # instance 1
        [[3, 4, 5], [0, 0, 0]], # instance 2
        [[6, 7, 8], [6, 5, 4]], # instance 3
        [[9, 0, 1], [3, 2, 1]], # instance 4
    ])

with tf.Session() as sess:
    init.run()
    outputs_val = sess.run(outputs, feed_dict={X: X_batch})

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [20]:
outputs_val

array([[[-0.85115266,  0.87358344,  0.5802911 ,  0.8954789 ,
         -0.0557505 ],
        [-0.999996  ,  0.99999577,  0.9981815 ,  1.        ,
          0.37679607]],

       [[-0.9983293 ,  0.9992038 ,  0.98071456,  0.999985  ,
          0.25192663],
        [-0.7081804 , -0.0772338 , -0.85227895,  0.5845349 ,
         -0.78780943]],

       [[-0.9999827 ,  0.99999535,  0.9992863 ,  1.        ,
          0.5159072 ],
        [-0.9993956 ,  0.9984095 ,  0.83422637,  0.99999976,
         -0.47325212]],

       [[ 0.87888587,  0.07356028,  0.97216916,  0.9998546 ,
         -0.7351168 ],
        [-0.9134514 ,  0.3600957 ,  0.7624866 ,  0.99817705,
          0.80142   ]]], dtype=float32)

# Setting the sequences length

In [21]:
reset_graph()

seq_length = tf.placeholder(tf.int32, [None])

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

X_batch = np.array([
        # t = 0      t = 1 
        [[0, 1, 2], [9, 8, 7]], # instance 1
        [[3, 4, 5], [0, 0, 0]], # instance 2
        [[6, 7, 8], [6, 5, 4]], # instance 3
        [[9, 0, 1], [3, 2, 1]], # instance 4
    ])

In [22]:
seq_length_batch = np.array([2, 1, 2, 2])

In [23]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run([outputs, states], 
                                       feed_dict={X: X_batch, seq_length: seq_length_batch})

In [24]:
outputs_val

array([[[ 0.35624826, -0.07799593,  0.82689077,  0.5365495 ,
         -0.27484432],
        [ 0.9999992 ,  0.5473382 ,  0.99835557, -0.9993405 ,
         -0.9999994 ]],

       [[ 0.9971697 , -0.15232755,  0.99498975, -0.67946047,
         -0.9895291 ],
        [-0.57477486,  0.35649702, -0.4767929 ,  0.7996426 ,
         -0.8029966 ]],

       [[ 0.99999166, -0.22497495,  0.9998668 , -0.97826475,
         -0.99990267],
        [ 0.99989593,  0.20977932,  0.9672055 , -0.9856287 ,
         -0.9999641 ]],

       [[ 0.9996179 ,  0.8324094 ,  0.8967202 , -0.9997168 ,
         -0.9946288 ],
        [ 0.985361  , -0.435984  ,  0.396685  , -0.774414  ,
         -0.9985339 ]]], dtype=float32)

In [25]:
states_val

array([[ 0.9999992 ,  0.5473382 ,  0.99835557, -0.9993405 , -0.9999994 ],
       [-0.57477486,  0.35649702, -0.4767929 ,  0.7996426 , -0.8029966 ],
       [ 0.99989593,  0.20977932,  0.9672055 , -0.9856287 , -0.9999641 ],
       [ 0.985361  , -0.435984  ,  0.396685  , -0.774414  , -0.9985339 ]],
      dtype=float32)

# Training RNNs

## Training a Sequence Classifier

In [32]:
reset_graph()

n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, 
                                                          logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [33]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [34]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [36]:
X_test = X_test.reshape((-1, n_steps, n_inputs))

In [39]:
n_epochs = 10
batch_size = 150

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Last batch accuracy:", acc_train, "Test accuracy:", acc_test)

0 Last batch accuracy: 0.93333334 Test accuracy: 0.9256
1 Last batch accuracy: 0.96666664 Test accuracy: 0.9509
2 Last batch accuracy: 0.94 Test accuracy: 0.9594
3 Last batch accuracy: 0.97333336 Test accuracy: 0.9622
4 Last batch accuracy: 0.96 Test accuracy: 0.9589
5 Last batch accuracy: 0.9866667 Test accuracy: 0.9703
6 Last batch accuracy: 0.97333336 Test accuracy: 0.9636
7 Last batch accuracy: 0.97333336 Test accuracy: 0.967
8 Last batch accuracy: 0.9866667 Test accuracy: 0.973
9 Last batch accuracy: 0.96 Test accuracy: 0.956


## Using `OutputProjectionWrapper()`

In [53]:
reset_graph()

n_setps = 20
n_inputs = 1
n_neurons = 100
n_outputs = 1

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])
cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [62]:
t_min, t_max = 0, 30
resolution = 0.1

def time_series(t):
    return t * np.sin(t) / 3 + 2 * np.sin(t*5)

def next_batch(batch_size, n_steps):
    t0 = np.random.rand(batch_size, 1) * (t_max - t_min - n_steps * resolution)
    Ts = t0 + np.arange(0., n_steps + 1) * resolution
    ys = time_series(Ts)
    return ys[:, :-1].reshape(-1, n_steps, 1), ys[:, 1:].reshape(-1, n_steps, 1)

t = np.linspace(t_min, t_max, int((t_max - t_min) / resolution))

n_steps = 20
t_instance = np.linspace(12.2, 12.2 + resolution * (n_steps + 1), n_steps + 1)

In [55]:
reset_graph()

n_setps = 20
n_inputs = 1
n_neurons = 100
n_outputs = 1

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

In [56]:
cell = tf.contrib.rnn.OutputProjectionWrapper(
    tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),
    output_size=n_outputs)

In [57]:
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

learning_rate = 0.001

loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_iterations = 1500
batch_size = 50

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        X_batch, y_batch= next_batch(batch_size, n_steps)
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if iteration % 100 == 0:
            mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print(iteration, "\tMSE:", mse)
    
    saver.save(sess, "./my_time_series_model")

0 	MSE: 10.666613
100 	MSE: 0.30682725
200 	MSE: 0.1033181
300 	MSE: 0.051026616
400 	MSE: 0.04019906
500 	MSE: 0.034831803
600 	MSE: 0.03198757
700 	MSE: 0.03482099
800 	MSE: 0.03513612
900 	MSE: 0.03594706
1000 	MSE: 0.030542314
1100 	MSE: 0.04036259
1200 	MSE: 0.027128106
1300 	MSE: 0.029048223
1400 	MSE: 0.029854676


## Without using `OutputProjectionWrapper()`

In [58]:
reset_graph()

n_steps = 20
n_inputs = 1
n_neurons = 100

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

In [59]:
cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
rnn_outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [60]:
stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons])
stacked_outputs = tf.layers.dense(stacked_rnn_outputs, n_outputs, 
                                  activation=None)  # just projection, without using any activation function
outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])

In [63]:
loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_iterations = 1500
batch_size = 50

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        X_batch, y_batch= next_batch(batch_size, n_steps)
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if iteration % 100 == 0:
            mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print(iteration, "\tMSE:", mse)
    
    X_new = time_series(np.array(t_instance[:-1].reshape(-1, n_steps, n_inputs)))
    y_pred = sess.run(outputs, feed_dict={X: X_new})

0 	MSE: 18.788918
100 	MSE: 0.5265741
200 	MSE: 0.3055803
300 	MSE: 0.07024149
400 	MSE: 0.06695416
500 	MSE: 0.053447407
600 	MSE: 0.042338837
700 	MSE: 0.04775281
800 	MSE: 0.055625413
900 	MSE: 0.045610733
1000 	MSE: 0.0429216
1100 	MSE: 0.040087167
1200 	MSE: 0.041430518
1300 	MSE: 0.03973811
1400 	MSE: 0.035608407


In [64]:
y_pred

array([[[-3.441709  ],
        [-2.5583212 ],
        [-1.2032008 ],
        [ 0.75186884],
        [ 2.214656  ],
        [ 3.1460192 ],
        [ 3.51014   ],
        [ 3.3311012 ],
        [ 2.8228183 ],
        [ 2.2012165 ],
        [ 1.7323577 ],
        [ 1.6036612 ],
        [ 1.9419342 ],
        [ 2.7414203 ],
        [ 3.881808  ],
        [ 5.1178226 ],
        [ 6.1614842 ],
        [ 6.6983504 ],
        [ 6.649885  ],
        [ 6.0500145 ]]], dtype=float32)